In [54]:
import yfinance as yf
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import pandas as pd
import numpy as np
import cvxpy as cp
from datetime import datetime, timedelta
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import nltk
from bs4 import BeautifulSoup
import requests

In [80]:

def fetch_stock_data(tickers):
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=15*365)).strftime('%Y-%m-%d')
    all_data = {}
    valid_tickers = []

    for ticker in tickers:
        try:
            df = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
            if df.isna().sum() > 0:
                available_start_date = df.first_valid_index()
                if available_start_date is not None:
                    df = yf.download(ticker, start=available_start_date.strftime('%Y-%m-%d'), end=end_date)['Adj Close']
            if not df.empty:
                all_data[ticker] = df
                valid_tickers.append(ticker)
            else:
                print(f"Data for {ticker} is empty.")
        except Exception as e:
            print(f"Failed to download data for {ticker}: {e}")

    if len(valid_tickers) == 0:
        raise ValueError("No valid tickers to optimize.")

    data_frame = pd.DataFrame(all_data)
    return data_frame.dropna(), valid_tickers

In [70]:
# Function to scrape news and analyze sentiment
def scrape_news_and_analyze_sentiment(tickers):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = {ticker: [] for ticker in tickers}

    for ticker in tickers:
        # Example URL, you may need to change it based on the website structure
        url = f"https://news.google.com/search?q={ticker}&hl=en-US&gl=US&ceid=US:en"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        one_month_ago = datetime.today() - timedelta(days=30)
        
        for article in soup.find_all('article'):
            title_tag = article.find('h3') or article.find('h2')
            if title_tag:
                title = title_tag.get_text()
                date_tag = article.find('time')
                if date_tag and date_tag.has_attr('datetime'):
                    date_str = date_tag['datetime']
                    date = datetime.fromisoformat(date_str[:-1])
                    if date >= one_month_ago:
                        sentiment = analyzer.polarity_scores(title)
                        sentiment_scores[ticker].append(sentiment['compound'])

    # Calculate average sentiment score for each ticker
    average_sentiment = {ticker: np.mean(scores) if scores else 0 for ticker, scores in sentiment_scores.items()}
    return average_sentiment

In [72]:
# Function to adjust expected returns based on sentiment
def adjust_returns_with_sentiment(returns, sentiment_scores):
    adjusted_returns = returns.copy()
    for stock in adjusted_returns.index:
        sentiment = sentiment_scores.get(stock, 0)
        adjusted_returns[stock] += sentiment * 0.01  # Adjust this factor as needed
    return adjusted_returns

In [74]:
#support fuc
def calculate_var(df, confidence_level=0.95):
    returns = df.pct_change().dropna()
    var = returns.quantile(1 - confidence_level)
    return var

def calculate_sharpe_ratio(df):
    returns = df.pct_change().dropna()
    sharpe_ratio = returns.mean() / returns.std() * np.sqrt(252)  # Annualized Sharpe Ratio
    return sharpe_ratio

In [92]:
import numpy as np
import cvxpy as cp
from pypfopt import EfficientFrontier, DiscreteAllocation, expected_returns, risk_models
from pypfopt.discrete_allocation import get_latest_prices

def optimize_portfolio(tickers, investment_amount, risk_preference):
    # Fetch historical stock prices
    df, valid_tickers = fetch_stock_data(tickers)
    
    # Calculate expected returns and sample covariance
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.CovarianceShrinkage(df).ledoit_wolf()
    
    # Calculate risk metrics for each stock
    var = calculate_var(df)
    sharpe_ratios = df.apply(calculate_sharpe_ratio)
    
    # Scrape news and analyze sentiment
    sentiment_scores = scrape_news_and_analyze_sentiment(valid_tickers)
    mu = adjust_returns_with_sentiment(mu, sentiment_scores)

    # Adjust returns based on risk preference
    if risk_preference == 'averse':
        mu -= var * 0.1  # Penalize high VaR stocks
    elif risk_preference == 'like':
        mu += sharpe_ratios * 0.1  # Favor high Sharpe Ratio stocks

    # Optimize for maximum Sharpe ratio
    ef = EfficientFrontier(mu, S)
    if risk_preference == 'averse':
        var_array = var.values  # Ensure var is a numpy array
        var_limit = np.percentile(var_array, 95)
        # Add constraint for VaR using cvxpy
        ef.add_constraint(lambda w: cp.sum(cp.multiply(var_array, w)) <= var_limit)
    weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    
    # Calculate allocation based on investment amount
    latest_prices = get_latest_prices(df)
    
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=investment_amount)
    allocation, leftover = da.lp_portfolio()
    
    
    # Display portfolio performance
    performance = ef.portfolio_performance(verbose=True)
    
    return cleaned_weights, allocation, leftover, performance


In [102]:
if __name__ == "__main__":
    # Input from user
    tickers = input("Enter stock tickers (comma separated): ").split(',')
    tickers = [ticker.strip() for ticker in tickers]  # Remove any extra whitespace
    investment_amount = float(input("Enter investment amount: "))
    
    # Input risk preference
    print("Select risk preference:")
    print("1. Risk Averse")
    print("2. Risk Neutral")
    print("3. Risk Seeking")
    risk_preference_input = int(input("Enter the number corresponding to your risk preference (1/2/3): "))
    
    risk_preference = 'neutral'
    if risk_preference_input == 1:
        risk_preference = 'averse'
    elif risk_preference_input == 3:
        risk_preference = 'like'

    try:
        weights, allocation, leftover, performance = optimize_portfolio(tickers, investment_amount, risk_preference)
        
        print("\nOptimal Weights:")
        for ticker, weight in weights.items():
            print(f"The cleaned weight for {ticker} is: {weight:.4f}")
        
        print("\nInvestment Allocation:")
        if allocation:
            for ticker, amount in allocation.items():
                print(f"The allocation for {ticker} is: {amount:.0f} stocks")
        else:
            print("No allocation was made.")
        
        print(f"\nFunds Remaining: ${leftover:.2f}")
        
        print("\nPortfolio Performance:")
        print(f"Expected annual return: {performance[0]:.2f}")
        print(f"Annual volatility: {performance[1]:.2f}")
        print(f"Sharpe ratio: {performance[2]:.2f}")
    except ValueError as e:
        print(f"Error: {e}")


Enter stock tickers (comma separated):  AAPL, MSFT, GOOGL, AMZN, TSLA, NFLX, NVDA, JPM, BAC
Enter investment amount:  100000


Select risk preference:
1. Risk Averse
2. Risk Neutral
3. Risk Seeking


Enter the number corresponding to your risk preference (1/2/3):  1


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Debug Info - Cleaned Weights: OrderedDict({'AAPL': 0.21403, 'MSFT': 0.12221, 'GOOGL': 0.0, 'AMZN': 0.11325, 'TSLA': 0.0938, 'NFLX': 0.01532, 'NVDA': 0.44139, 'JPM': 0.0, 'BAC': 0.0})
Debug Info - Latest Prices: AAPL     227.570007
MSFT     454.700012
GOOGL    185.570007
AMZN     195.050003
TSLA     241.029999
NFLX     652.750000
NVDA     127.400002
JPM      207.449997
BAC       41.810001
Name: 2024-07-11 00:00:00, dtype: float64
Debug Info - Allocation: {'AAPL': 94, 'MSFT': 27, 'AMZN': 58, 'TSLA': 39, 'NFLX': 2, 'NVDA': 347}
Debug Info - Leftover: 105.1482839124219
Expected annual return: 42.1%
Annual volatility: 30.6%
Sharpe Ratio: 1.31

Optimal Weights:
The cleaned weight for AAPL is: 0.2140
The cleaned weight for MSFT is: 0.1222
The cleaned weight for GOOGL is: 0.0000
The cleaned weight for AMZN is: 0.1133
The cleaned weight for TSLA is: 0.0938
The cleaned weight for NFLX is: 0.0153
The cleaned weight for NVDA is: 0.4414
The cleaned weight for JPM is: 0.0000
The cleaned weight for B

In [37]:
# # Function to run the portfolio optimization
# def run_optimization():
#     tickers = entry_tickers.get().split(',')
#     tickers = [ticker.strip() for ticker in tickers]
#     investment_amount = float(entry_amount.get())
    
#     try:
#         weights, allocation, leftover, performance = optimize_portfolio(tickers, investment_amount)
        
#         # Display the results in a message box
#         result_message = "\nOptimal Weights:\n"
#         for ticker, weight in weights.items():
#             result_message += f"The cleaned weight for {ticker} is: {weight:.4f}\n"
        
#         result_message += "\nInvestment Allocation:\n"
#         for ticker, amount in allocation.items():
#             result_message += f"The allocation for {ticker} is: ${amount:.2f}\n"
        
#         result_message += f"\nFunds Remaining: ${leftover:.2f}\n"
        
#         result_message += "\nPortfolio Performance:\n"
#         result_message += f"Expected annual return: {performance[0]:.2f}\n"
#         result_message += f"Annual volatility: {performance[1]:.2f}\n"
#         result_message += f"Sharpe ratio: {performance[2]:.2f}\n"
        
#         messagebox.showinfo("Optimization Result", result_message)
#     except Exception as e:
#         messagebox.showerror("Error", str(e))

# # Create the main window
# root = tk.Tk()
# root.title("Portfolio Optimization")

# # Create and place the widgets
# tk.Label(root, text="Enter stock tickers (comma separated):").grid(row=0, column=0)
# entry_tickers = tk.Entry(root)
# entry_tickers.grid(row=0, column=1)

# tk.Label(root, text="Enter investment amount:").grid(row=1, column=0)
# entry_amount = tk.Entry(root)
# entry_amount.grid(row=1, column=1)

# tk.Button(root, text="Optimize Portfolio", command=run_optimization).grid(row=2, columnspan=2)

# # Start the main event loop
# root.mainloop()


In [39]:
# def run_optimization():
#     tickers = entry_tickers.get().split(',')
#     tickers = [ticker.strip() for ticker in tickers]
#     investment_amount = float(entry_amount.get())
    
#     try:
#         weights, allocation, leftover, performance = optimize_portfolio(tickers, investment_amount)
        
#         # Display the results in a message box
#         result_message = "\nOptimal Weights:\n"
#         for ticker, weight in weights.items():
#             result_message += f"The cleaned weight for {ticker} is: {weight:.4f}\n"
        
#         result_message += "\nInvestment Allocation:\n"
#         for ticker, amount in allocation.items():
#             result_message += f"The allocation for {ticker} is: ${amount:.2f}\n"
        
#         result_message += f"\nFunds Remaining: ${leftover:.2f}\n"
        
#         result_message += "\nPortfolio Performance:\n"
#         result_message += f"Expected annual return: {performance[0]:.2f}\n"
#         result_message += f"Annual volatility: {performance[1]:.2f}\n"
#         result_message += f"Sharpe ratio: {performance[2]:.2f}\n"
        
#         messagebox.showinfo("Optimization Result", result_message)
#     except Exception as e:
#         messagebox.showerror("Error", str(e))

In [41]:
# root = tk.Tk()
# root.title("Portfolio Optimization")

# tk.Label(root, text="Enter stock tickers (comma separated):").grid(row=0, column=0)
# entry_tickers = tk.Entry(root)
# entry_tickers.grid(row=0, column=1)

# tk.Label(root, text="Enter investment amount:").grid

<bound method Grid.grid_configure of <tkinter.Label object .!label2>>

In [104]:
pip install voila

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 21.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 14.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
